In [ ]:
#!/usr/bin/env python3
"""
BC Parks Day Pass Availability Notifier
Monitors BC Parks day pass availability and sends Telegram notifications
"""

import requests
import time
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import schedule
from bs4 import BeautifulSoup
import sqlite3
import os

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('bc_parks_monitor.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class BCParksMonitor:
    def __init__(self, telegram_bot_token: str, telegram_chat_id: str, target_parks: List[str] = None):
        self.telegram_bot_token = telegram_bot_token
        self.telegram_chat_id = telegram_chat_id
        self.base_url = "https://reserve.bcparks.ca"
        self.dayuse_url = f"{self.base_url}/dayuse/registration"
        self.target_parks = target_parks or ['joffre', 'joffrey']  # Default to Joffrey Lakes variations
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        self.db_path = 'bc_parks_availability.db'
        self.init_database()

    def init_database(self):
        """Initialize SQLite database to track availability changes"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS availability (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                park_name TEXT,
                date TEXT,
                available_spots INTEGER,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                notified BOOLEAN DEFAULT FALSE
            )
        ''')
        conn.commit()
        conn.close()

    def send_telegram_message(self, message: str) -> bool:
        """Send message via Telegram bot"""
        try:
            url = f"https://api.telegram.org/bot{self.telegram_bot_token}/sendMessage"
            payload = {
                'chat_id': self.telegram_chat_id,
                'text': message,
                'parse_mode': 'HTML'
            }
            response = requests.post(url, json=payload, timeout=10)
            response.raise_for_status()
            logger.info("Telegram message sent successfully")
            return True
        except Exception as e:
            logger.error(f"Failed to send Telegram message: {e}")
            return False

    def get_available_parks(self) -> List[Dict]:
        """Scrape BC Parks website for available day passes, focusing on Joffrey Lakes"""
        try:
            logger.info("Fetching BC Parks day use page...")

            # First try Joffrey-specific approaches
            joffrey_data = self.get_joffrey_specific_data()
            if joffrey_data:
                logger.info("Found Joffrey Lakes data via direct approach")
                return joffrey_data

            # Fall back to general scraping but filter for Joffrey
            response = self.session.get(self.dayuse_url, timeout=15)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            parks_data = []

            # Look for park information containers
            park_containers = soup.find_all('div', class_=['park-item', 'reservation-item', 'facility-item'])

            if not park_containers:
                # Try alternative selectors
                park_containers = soup.find_all('div', attrs={'data-park-id': True})

            if not park_containers:
                # Try finding any div containing "joffre" or "joffrey"
                park_containers = soup.find_all('div', text=lambda text: text and any(
                    target.lower() in text.lower() for target in self.target_parks
                ))

            for container in park_containers:
                try:
                    park_info = self.extract_park_info(container)
                    if park_info and self.is_target_park(park_info['name']):
                        parks_data.append(park_info)
                        logger.info(f"Found target park: {park_info['name']}")
                except Exception as e:
                    logger.warning(f"Error parsing park container: {e}")
                    continue

            # If no specific containers found, try searching the entire page for Joffrey references
            if not parks_data:
                joffrey_links = soup.find_all('a', href=lambda href: href and any(
                    target.lower() in href.lower() for target in self.target_parks
                ))

                for link in joffrey_links:
                    try:
                        href = link.get('href')
                        full_url = href if href.startswith('http') else f"{self.base_url}{href}"

                        # Fetch the Joffrey-specific page
                        joffrey_response = self.session.get(full_url, timeout=15)
                        if joffrey_response.status_code == 200:
                            joffrey_soup = BeautifulSoup(joffrey_response.content, 'html.parser')
                            joffrey_info = self.parse_joffrey_page(joffrey_soup, full_url)
                            if joffrey_info:
                                parks_data.append(joffrey_info)
                                break
                    except Exception as e:
                        logger.warning(f"Error fetching Joffrey page from link: {e}")
                        continue

            logger.info(f"Found {len(parks_data)} Joffrey Lakes entries")
            return parks_data

        except Exception as e:
            logger.error(f"Error fetching park data: {e}")
            return []

    def is_target_park(self, park_name: str) -> bool:
        """Check if park matches our target parks (Joffrey Lakes)"""
        park_name_lower = park_name.lower()
        return any(target.lower() in park_name_lower for target in self.target_parks)

    def get_joffrey_specific_data(self) -> List[Dict]:
        """Get Joffrey Lakes specific data with multiple approaches"""
        try:
            # Try direct Joffrey Lakes URL patterns
            joffrey_urls = [
                f"{self.base_url}/dayuse/registration?facility=joffre-lakes",
                f"{self.base_url}/dayuse/registration?park=joffre-lakes",
                f"{self.base_url}/facility/joffre-lakes-provincial-park",
                f"{self.base_url}/dayuse/joffre-lakes"
            ]

            for url in joffrey_urls:
                try:
                    logger.info(f"Trying Joffrey-specific URL: {url}")
                    response = self.session.get(url, timeout=15)
                    if response.status_code == 200:
                        soup = BeautifulSoup(response.content, 'html.parser')
                        joffrey_data = self.parse_joffrey_page(soup, url)
                        if joffrey_data:
                            return [joffrey_data]
                except Exception as e:
                    logger.debug(f"Failed to fetch {url}: {e}")
                    continue

            return []

        except Exception as e:
            logger.error(f"Error in Joffrey-specific data fetch: {e}")
            return []

    def parse_joffrey_page(self, soup: BeautifulSoup, url: str) -> Optional[Dict]:
        """Parse Joffrey Lakes specific page"""
        try:
            # Look for availability information
            availability_indicators = [
                soup.find(text=lambda text: text and 'available' in text.lower()),
                soup.find('span', class_=['available', 'spots']),
                soup.find('div', class_=['availability', 'booking-info']),
            ]

            available_spots = 0
            for indicator in availability_indicators:
                if indicator:
                    text = str(indicator).lower()
                    if 'sold out' in text or 'full' in text:
                        available_spots = 0
                        break
                    elif 'available' in text:
                        # Try to extract number
                        numbers = [int(s) for s in str(indicator).split() if s.isdigit()]
                        available_spots = numbers[0] if numbers else 1
                        break

            # Look for booking button or link
            book_elements = soup.find_all(['a', 'button'], text=lambda text: text and any(
                word in text.lower() for word in ['book', 'reserve', 'select']
            ))

            booking_url = url
            for elem in book_elements:
                if elem.get('href'):
                    href = elem['href']
                    booking_url = href if href.startswith('http') else f"{self.base_url}{href}"
                    break

            return {
                'name': 'Joffre Lakes Provincial Park',
                'available_spots': available_spots,
                'booking_url': booking_url,
                'source_url': url
            }

        except Exception as e:
            logger.warning(f"Error parsing Joffrey page: {e}")
            return None
        """Extract park information from HTML container"""
        try:
            # Extract park name
            name_elem = container.find(['h3', 'h2', 'h4']) or container.find(class_=['park-name', 'facility-name'])
            park_name = name_elem.get_text(strip=True) if name_elem else "Unknown Park"

            # Look for availability indicators
            availability_elem = container.find(class_=['available', 'spots-available', 'availability'])
            available_spots = 0

            if availability_elem:
                # Try to extract number from text
                text = availability_elem.get_text(strip=True)
                numbers = [int(s) for s in text.split() if s.isdigit()]
                if numbers:
                    available_spots = numbers[0]
                elif 'available' in text.lower():
                    available_spots = 1  # Available but no specific count

            # Look for booking links or buttons
            book_link = container.find('a', href=True)
            booking_url = ""
            if book_link and book_link.get('href'):
                href = book_link['href']
                booking_url = href if href.startswith('http') else f"{self.base_url}{href}"

            return {
                'name': park_name,
                'available_spots': available_spots,
                'booking_url': booking_url,
                'raw_html': str(container)[:200] + "..." if len(str(container)) > 200 else str(container)
            }

        except Exception as e:
            logger.warning(f"Error extracting park info: {e}")
            return None

    def check_api_endpoints(self) -> List[Dict]:
        """Try to find and query API endpoints for more reliable data"""
        try:
            # Common API endpoint patterns
            api_endpoints = [
                f"{self.base_url}/api/dayuse/availability",
                f"{self.base_url}/api/facilities/availability",
                f"{self.base_url}/dayuse/api/availability"
            ]

            for endpoint in api_endpoints:
                try:
                    response = self.session.get(endpoint, timeout=10)
                    if response.status_code == 200:
                        data = response.json()
                        logger.info(f"Found API endpoint: {endpoint}")
                        return self.parse_api_data(data)
                except Exception:
                    continue

        except Exception as e:
            logger.warning(f"API endpoint check failed: {e}")

        return []

    def parse_api_data(self, data: Dict) -> List[Dict]:
        """Parse API response data"""
        parks_data = []
        try:
            # Adapt this based on actual API structure
            if isinstance(data, dict):
                if 'facilities' in data:
                    for facility in data['facilities']:
                        parks_data.append({
                            'name': facility.get('name', 'Unknown'),
                            'available_spots': facility.get('available_spots', 0),
                            'booking_url': facility.get('booking_url', ''),
                        })
                elif 'parks' in data:
                    for park in data['parks']:
                        parks_data.append({
                            'name': park.get('name', 'Unknown'),
                            'available_spots': park.get('availability', 0),
                            'booking_url': park.get('url', ''),
                        })
        except Exception as e:
            logger.error(f"Error parsing API data: {e}")

        return parks_data

    def store_availability(self, parks_data: List[Dict]):
        """Store availability data in database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        current_date = datetime.now().strftime('%Y-%m-%d')

        for park in parks_data:
            cursor.execute('''
                INSERT INTO availability (park_name, date, available_spots)
                VALUES (?, ?, ?)
            ''', (park['name'], current_date, park['available_spots']))

        conn.commit()
        conn.close()

    def get_previous_availability(self, park_name: str) -> Optional[int]:
        """Get previous availability for comparison"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            SELECT available_spots FROM availability
            WHERE park_name = ?
            ORDER BY timestamp DESC
            LIMIT 1
        ''', (park_name,))

        result = cursor.fetchone()
        conn.close()

        return result[0] if result else None

    def check_availability(self):
        """Main method to check Joffrey Lakes availability and send notifications"""
        logger.info("Starting Joffrey Lakes availability check...")

        # Try API first, fall back to web scraping
        parks_data = self.check_api_endpoints()
        if not parks_data:
            parks_data = self.get_available_parks()

        # Filter for Joffrey Lakes if we got general data
        if parks_data:
            parks_data = [park for park in parks_data if self.is_target_park(park['name'])]

        if not parks_data:
            logger.warning("No Joffrey Lakes data retrieved")
            # Send a notification if we consistently can't find data
            self.send_no_data_notification()
            return

        notifications = []

        for park in parks_data:
            park_name = park['name']
            current_spots = park['available_spots']
            previous_spots = self.get_previous_availability(park_name)

            # Check for new availability
            if current_spots > 0:
                if previous_spots is None or previous_spots == 0:
                    # New availability found
                    message = f"🏔️ <b>JOFFRE LAKES DAY PASS AVAILABLE!</b> 🎉\n\n"
                    message += f"📍 <b>Location:</b> Joffre Lakes Provincial Park\n"
                    message += f"🎫 <b>Available Spots:</b> {current_spots}\n"
                    message += f"⏰ <b>Time Found:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
                    if park.get('booking_url'):
                        message += f"🔗 <b>BOOK NOW:</b> {park['booking_url']}\n"
                    message += f"\n💨 <b>Book quickly - spots fill up fast!</b>"

                    notifications.append(message)

                elif current_spots > previous_spots:
                    # Increased availability
                    message = f"📈 <b>MORE JOFFRE LAKES SPOTS!</b> 🏔️\n\n"
                    message += f"📍 <b>Location:</b> Joffre Lakes Provincial Park\n"
                    message += f"🎫 <b>Available Spots:</b> {current_spots} (+{current_spots - previous_spots} new)\n"
                    message += f"⏰ <b>Updated:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
                    if park.get('booking_url'):
                        message += f"🔗 <b>BOOK NOW:</b> {park['booking_url']}\n"
                    message += f"\n💨 <b>Additional spots just opened up!</b>"

                    notifications.append(message)
            else:
                # No spots available - only log, don't notify unless it's a change
                if previous_spots is None or previous_spots > 0:
                    logger.info("Joffre Lakes is currently sold out")

        # Store current availability
        self.store_availability(parks_data)

        # Send notifications
        for notification in notifications:
            self.send_telegram_message(notification)
            time.sleep(1)  # Rate limiting

        if notifications:
            logger.info(f"Sent {len(notifications)} Joffre Lakes notifications")
        else:
            logger.info("No new Joffre Lakes availability to notify about")

    def send_no_data_notification(self):
        """Send notification when we can't find Joffrey Lakes data"""
        # Only send this once per day to avoid spam
        last_check = datetime.now().strftime('%Y-%m-%d')

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            SELECT COUNT(*) FROM availability
            WHERE park_name = 'NO_DATA_FOUND' AND date = ?
        ''', (last_check,))

        if cursor.fetchone()[0] == 0:
            message = f"⚠️ <b>Joffre Lakes Monitor Status</b>\n\n"
            message += f"Unable to find Joffre Lakes data on BC Parks website.\n"
            message += f"This could mean:\n"
            message += f"• Website structure changed\n"
            message += f"• Reservations not yet open\n"
            message += f"• Temporary site issues\n\n"
            message += f"Monitor will keep checking every 15 minutes."

            self.send_telegram_message(message)

            # Record that we sent this notification
            cursor.execute('''
                INSERT INTO availability (park_name, date, available_spots)
                VALUES (?, ?, ?)
            ''', ('NO_DATA_FOUND', last_check, 0))
            conn.commit()

        conn.close()

    def send_daily_summary(self):
        """Send daily summary focused on Joffrey Lakes"""
        parks_data = self.check_api_endpoints()
        if not parks_data:
            parks_data = self.get_available_parks()

        # Filter for Joffrey Lakes
        joffrey_parks = [park for park in parks_data if self.is_target_park(park['name'])]

        message = f"🏔️ <b>Joffre Lakes Daily Summary</b>\n"
        message += f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n"

        if joffrey_parks:
            available_joffrey = [park for park in joffrey_parks if park['available_spots'] > 0]

            if available_joffrey:
                message += f"✅ <b>DAY PASSES AVAILABLE!</b>\n"
                for park in available_joffrey:
                    message += f"🎫 {park['available_spots']} spots at {park['name']}\n"
                    if park.get('booking_url'):
                        message += f"🔗 {park['booking_url']}\n"
            else:
                message += f"❌ <b>Currently sold out</b>\n"
                message += f"Monitor will alert you when spots become available"
        else:
            message += f"⚠️ <b>No data found</b>\n"
            message += f"Will keep monitoring for updates"

        message += f"\n🔄 Checking every 15 minutes for availability"

        self.send_telegram_message(message)

def main():
    # Configuration - Replace with your actual values OR use environment variables
    TELEGRAM_BOT_TOKEN = os.getenv('TELEGRAM_BOT_TOKEN', "8298235873:AAEItS-3RBmrrtSuGWFXkTN2rN_a8T9VXvw")
    TELEGRAM_CHAT_ID = os.getenv('TELEGRAM_CHAT_ID', "1295185522")

    # Test connection first
    print(f"🔍 Testing Telegram connection...")
    print(f"Bot Token: {TELEGRAM_BOT_TOKEN[:10]}...{TELEGRAM_BOT_TOKEN[-10:]}")
    print(f"Chat ID: {TELEGRAM_CHAT_ID}")

    # Validate credentials format
    if not TELEGRAM_BOT_TOKEN or ':' not in TELEGRAM_BOT_TOKEN:
        print("❌ Invalid bot token format")
        return


    # Initialize monitor specifically for Joffrey Lakes
    # You can modify target_parks to include variations of the name
    target_parks = ['joffre', 'joffrey', 'joffre lakes', 'joffrey lakes']

    # Test the connection before starting
    monitor = BCParksMonitor(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, target_parks)

    print("📡 Testing Telegram connection...")
    test_message = "🧪 <b>Connection Test</b>\n\nIf you see this message, your Joffre Lakes monitor is configured correctly!"

    if monitor.send_telegram_message(test_message):
        print("✅ Telegram connection successful!")
    else:
        print("❌ Telegram connection failed. Please check your credentials.")
        return

    # Schedule more frequent checks for Joffrey Lakes (every 10 minutes)
    schedule.every(10).minutes.do(monitor.check_availability)
    schedule.every().day.at("07:00").do(monitor.send_daily_summary)  # Daily summary at 7 AM
    schedule.every().day.at("19:00").do(monitor.send_daily_summary)  # Evening summary at 7 PM

    logger.info("Joffre Lakes Monitor started. Checking every 10 minutes...")

    # Send startup notification
    startup_message = f"🏔️ <b>Joffre Lakes Monitor Started!</b>\n\n"
    startup_message += f"🔍 Now monitoring Joffre Lakes Provincial Park\n"
    startup_message += f"⏰ Checking every 10 minutes\n"
    startup_message += f"📧 You'll get instant alerts when day passes become available\n\n"
    startup_message += f"💡 <b>Tip:</b> Joffre Lakes spots fill up very quickly, so be ready to book immediately!"

    monitor.send_telegram_message(startup_message)

    # Run initial check
    monitor.check_availability()

    # Keep running
    try:
        while True:
            schedule.run_pending()
            time.sleep(30)  # Check every 30 seconds for more responsive scheduling
    except KeyboardInterrupt:
        logger.info("Joffre Lakes Monitor stopped by user")
        monitor.send_telegram_message("⏹️ Joffre Lakes Monitor stopped")

if __name__ == "__main__":
    main()

🔍 Testing Telegram connection...
Bot Token: 8298235873...N_a8T9VXvw
Chat ID: 1295185522
📡 Testing Telegram connection...
✅ Telegram connection successful!


/tmp/ipython-input-2017667490.py:189: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  soup.find(text=lambda text: text and 'available' in text.lower()),
/tmp/ipython-input-2017667490.py:208: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  book_elements = soup.find_all(['a', 'button'], text=lambda text: text and any(


In [1]:
%pip install schedule